In [34]:
import os 
import warnings

warnings.filterwarnings("ignore")
os.chdir("/Users/erjo3868/repos/hypedsearch/hypedsearch")
print(f"cwd = {os.getcwd()}")

from src.lookups.protein_kmer_db import (
    ProteinKmerDb,
    prepare_protein_kmer_database
)
import time
from src.erik import (
    query_database,
    parse_mzml,
    get_indices_of_largest_elements
)
import numpy as np

cwd = /Users/erjo3868/repos/hypedsearch/hypedsearch


In [26]:
mzml_path = "data/spectra/BMEM_AspN_Fxn4.mzML"
spectra = list(parse_mzml(mzml_path=mzml_path))

In [ ]:
fasta_path = "tests/test.fasta"
db_path = "dbs/test.fasta.db"
max_kmer_len = 50
charges_to_consider = [1, 2, 3]
top_n_most_abundant = 50

t0 = time.time()
db = prepare_protein_kmer_database(
    db_path=db_path,
    fasta_path=fasta_path,
    max_kmer_len=max_kmer_len,
    charges_to_consider=charges_to_consider
)
t1 = time.time()
duration = round(t1 - t0, 2)
print(f"Preparing the database took {duration} secs = {round(duration/60, 2)} mins")

# query = """SELECT name
# FROM sqlite_master
# WHERE type = 'table';"""
# t0 = time.time()
# query_database(query=query, db_path=db_path)
# t1 = time.time()

# query = f"PRAGMA table_info(proteins);"
# query_database(query=query, db_path=db_path)

# query = f"PRAGMA table_info(kmers);"
# query_database(query=query, db_path=db_path)

for spectrum in spectra:
    # Look at most abundant 
    abundances = [peak.abundance for peak in spectrum.peaks]
    indices = get_indices_of_largest_elements(array=abundances, top_n=top_n_most_abundant)
    peaks = np.array(spectrum.peaks)
    top_n_peaks = peaks[indices]

    # 
    for peak in top_n_peaks:
    


    break


Preparing the database took 0.02 secs = 0.0 mins


50

In [33]:
abundances[[1,2]]

TypeError: list indices must be integers or slices, not list